In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam

In [4]:
# データの読み込み
data = pd.read_excel('data.xlsx',header = 1)

In [6]:
# 説明変数と目的変数の分離
X = data[['Concentration', 'time', 'tea']]

In [7]:
# tea変数のOneHotエンコード（BT, GT, GC）でBCは[0,0,0]に設定
def custom_onehot_encoding(df):
    onehot = pd.get_dummies(df['tea'])
    for col in ['BT', 'GT', 'GC']:
        if col not in onehot.columns:
            onehot[col] = 0
    onehot = onehot[['BT', 'GT', 'GC']]
    return pd.concat([df[['Concentration', 'time']], onehot], axis=1)

X_encoded = custom_onehot_encoding(X)

In [8]:
X_encoded 

,Concentration,time,BT,GT,GC
0,1,2,1,0,0
1,1,2,1,0,0
2,1,5,1,0,0
3,1,5,1,0,0
4,1,10,1,0,0
...,...,...,...,...,...
163,3,30,0,0,1
164,3,45,0,0,1
165,3,45,0,0,1
166,3,60,0,0,1


In [9]:
# 各ミネラルごとの回帰分析
minerals = ['Al', 'Ca', 'Cu', 'Fe', 'K', 'Mg', 'Mn', 'Na', 'Zn']

In [10]:
for mineral in minerals:
    print(f"\n### Mineral: {mineral} ###")
    
    # 目的変数を選択
    y = data[mineral]
    
    # データの分割
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
    
    # MLRモデルの構築と評価
    mlr_model = LinearRegression()
    mlr_model.fit(X_train, y_train)
    y_pred_mlr = mlr_model.predict(X_test)
    
    # 回帰係数の出力
    intercept = mlr_model.intercept_
    coefficients = mlr_model.coef_
    
    # 回帰式の表示
    feature_names = X_encoded.columns
    equation = f"{mineral} = {intercept:.4f} + " + " + ".join([f"({coef:.4f} * {name})" for coef, name in zip(coefficients, feature_names)])
    print("Regression Equation (MLR):")
    print(equation)
    
    # 評価指標の計算（MLR）
    r2_mlr = r2_score(y_test, y_pred_mlr)
    rmse_mlr = np.sqrt(mean_squared_error(y_test, y_pred_mlr))
    mae_mlr = mean_absolute_error(y_test, y_pred_mlr)
    
    print("\nMLR Model Performance:")
    print(f"R2: {r2_mlr:.4f}")
    print(f"RMSE: {rmse_mlr:.4f}")
    print(f"MAE: {mae_mlr:.4f}")


### Mineral: Al ###
Regression Equation (MLR):
Al = -6.6979 + (4.6421 * Concentration) + (0.0712 * time) + (5.5955 * BT) + (6.9472 * GT) + (3.4446 * GC)

MLR Model Performance:
R2: 0.7861
RMSE: 2.0952
MAE: 1.6406

### Mineral: Ca ###
Regression Equation (MLR):
Ca = 2.3143 + (1.0145 * Concentration) + (0.0252 * time) + (1.3988 * BT) + (0.8128 * GT) + (2.1923 * GC)

MLR Model Performance:
R2: 0.9317
RMSE: 0.3165
MAE: 0.2458

### Mineral: Cu ###
Regression Equation (MLR):
Cu = -0.0022 + (0.0195 * Concentration) + (0.0005 * time) + (0.0054 * BT) + (0.0019 * GT) + (0.0467 * GC)

MLR Model Performance:
R2: 0.6022
RMSE: 0.0230
MAE: 0.0171

### Mineral: Fe ###
Regression Equation (MLR):
Fe = -0.0415 + (0.0381 * Concentration) + (0.0016 * time) + (0.0946 * BT) + (0.0832 * GT) + (0.0944 * GC)

MLR Model Performance:
R2: 0.7746
RMSE: 0.0246
MAE: 0.0204

### Mineral: K ###
Regression Equation (MLR):
K = 38.1862 + (103.6673 * Concentration) + (1.9220 * time) + (-24.5165 * BT) + (-69.5713 * GT) + (

In [11]:
for mineral in minerals:
    print(f"\n### Mineral: {mineral} ###")
    
    # 目的変数を選択
    y = data[mineral]
    
    # データの分割
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
    
    # ANNモデルの構築
    ann_model = Sequential()
    ann_model.add(Dense(18, input_dim=X_train.shape[1], kernel_initializer='he_uniform'))
    ann_model.add(LeakyReLU(alpha=0.01))
    ann_model.add(Dense(8, kernel_initializer='he_uniform'))
    ann_model.add(LeakyReLU(alpha=0.01))
    ann_model.add(Dense(12, kernel_initializer='he_uniform'))
    ann_model.add(LeakyReLU(alpha=0.01))
    ann_model.add(Dense(18, kernel_initializer='he_uniform'))
    ann_model.add(LeakyReLU(alpha=0.01))
    ann_model.add(Dense(1, activation='linear'))
    
    # モデルのコンパイル
    ann_model.compile(optimizer=Adam(learning_rate=0.0112), loss='mse')
    
    # ANNモデルのトレーニング
    ann_model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)
    
    # ANNモデルの予測と評価
    y_pred_ann = ann_model.predict(X_test)
    
    # 評価指標の計算（ANN）
    r2_ann = r2_score(y_test, y_pred_ann)
    rmse_ann = np.sqrt(mean_squared_error(y_test, y_pred_ann))
    mae_ann = mean_absolute_error(y_test, y_pred_ann)
    
    print("\nANN Model Performance:")
    print(f"R2: {r2_ann:.4f}")
    print(f"RMSE: {rmse_ann:.4f}")
    print(f"MAE: {mae_ann:.4f}")


### Mineral: Al ###
2/2 [==============================] - 0s 2ms/step

ANN Model Performance:
R2: 0.9879
RMSE: 0.4985
MAE: 0.4037

### Mineral: Ca ###
2/2 [==============================] - 0s 2ms/step

ANN Model Performance:
R2: 0.8647
RMSE: 0.4454
MAE: 0.3598

### Mineral: Cu ###
2/2 [==============================] - 0s 4ms/step

ANN Model Performance:
R2: 0.6466
RMSE: 0.0217
MAE: 0.0172

### Mineral: Fe ###
2/2 [==============================] - 0s 2ms/step

ANN Model Performance:
R2: 0.0752
RMSE: 0.0498
MAE: 0.0330

### Mineral: K ###
2/2 [==============================] - 0s 3ms/step

ANN Model Performance:
R2: 0.9490
RMSE: 21.2391
MAE: 15.8817

### Mineral: Mg ###
2/2 [==============================] - 0s 3ms/step

ANN Model Performance:
R2: 0.9036
RMSE: 2.2662
MAE: 1.7430

### Mineral: Mn ###
2/2 [==============================] - 0s 3ms/step

ANN Model Performance:
R2: 0.8972
RMSE: 0.8843
MAE: 0.6722

### Mineral: Na ###
2/2 [==============================] - 0s 3ms/step

AN